# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
file = 'output_data/cities.csv'
cities = pd.read_csv(file)
cities

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Kapaa,22.08,-159.32,75.200,0.88,0.90,10.80,US,1594707408
1,Tasiilaq,65.61,-37.64,41.000,0.86,0.38,4.10,GL,1594707423
2,Rikitea,-23.12,-134.97,72.788,0.84,1.00,7.15,PF,1594707407
3,Isangel,-19.55,169.27,78.800,0.69,0.75,2.60,VU,1594707440
4,Lagoa,39.05,-27.98,67.964,0.78,0.13,2.53,PT,1594707588
...,...,...,...,...,...,...,...,...,...
560,Fortuna,40.60,-124.16,53.006,0.93,0.01,0.27,US,1594707427
561,Takoradze,4.88,-1.76,73.796,0.88,0.25,4.43,GH,1594707728
562,Farim,12.48,-15.22,75.398,0.93,0.96,2.23,GW,1594707728
563,Pundaguitan,6.37,126.17,84.632,0.66,0.05,1.72,PH,1594707728


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
gmaps.configure(api_key=g_key)
locations = cities[["Latitude", "Longitude"]].astype(float)
humidity = cities["Humidity (%)"].astype(float)

fig = gmaps.figure(center=(0, 0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=False, 
    max_intensity=1,
    point_radius=1
    )

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
#Narrow down the cities to fit weather conditions.
ideal_weather = cities[(cities['Temperature (F)'] > 70) & (cities['Humidity (%)'] < .5)].reset_index()
#Drop any rows will null values.
ideal_weather.dropna(inplace = True)
ideal_weather

,index,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,8,Saskylakh,71.92,114.08,74.696,0.34,0.45,3.92,RU,1594707124
1,18,Khatanga,71.97,102.50,74.048,0.38,0.89,2.94,RU,1594707413
2,29,Ekibastuz,51.67,75.37,80.726,0.43,0.03,4.96,KZ,1594707594
3,41,Yerbogachën,61.28,108.01,79.268,0.43,0.78,2.01,RU,1594707597
4,46,Aksu,41.12,80.26,81.806,0.31,0.98,2.11,CN,1594707480
...,...,...,...,...,...,...,...,...,...,...
67,522,Faya,18.39,42.45,75.200,0.29,0.13,2.60,SA,1594707717
68,548,Verkhoyansk,67.54,133.38,73.508,0.41,0.85,8.20,RU,1594707724
69,549,Ubari,26.59,12.78,75.920,0.36,0.00,4.55,LY,1594707725
70,550,Qandala,11.47,49.87,92.804,0.46,0.61,2.30,SO,1594707726


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
#Store into variable named hotel_df.
hotel_df = ideal_weather

#Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lng'] = ""

#Set parameters to search for hotels with 5000 meters.
radius = 5000

In [70]:
#Hit the Google Places API for each city's coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    lat = row['Latitude']
    lng = row['Longitude'] 
    
    params = {
    "location": f'{lat},{lng}',
    "radius": radius,
    "type": "lodging",
    "key": g_key,
    }

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #Store the first Hotel result into the DataFrame.j
    try:        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Lng'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = 'None found'
        pass

hotel_df = hotel_df[hotel_df['Hotel Name'] != 'None found'].head(10)

In [71]:
#Plot markers on top of the heatmap.
gmaps.configure(api_key=g_key)
hotel_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [74]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(hotel_markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))